Скачаем xlsx с сайте https://data.mos.ru/opendata/7710071979-kovorkingi

In [1]:
import requests 
import pandas as pd 
from bs4 import BeautifulSoup 
import urllib

In [2]:
coworking = pd.read_excel('data-4209-2017-11-13.xlsx')
coworking.head()

,Код,Полное название,global_id,Краткое название,Краткая характеристика,Административный округ,Район,Адрес,Уточнение местоположения,Информация о ценах,Год создания,Отрасль,Контактный телефон,Адрес электронной почты,Сайт,Геоданные
0,19,Коворкинг Union Place,4066280,Union Place,"Конференц-зал на 60 человек, переговорные комн...",Центральный административный округ,Таганский район,"1-й Гончарный переулок, д.3, стр.4","1-й Гончарный переулок, д.3, стр.4",Стоимость аренды конференц-зала 1500 рублей в ...,NaN,NaN,(495) 795-43-76,info@unionplace.ru,www.unionplace.ru,"{type=Point, coordinates=[37.65248776855666, 5..."
1,20,Коворкинг и клубный офис клуба предпринимателе...,4066281,Коворкинг Деловар,"Клуб создан в 2003 году, объединяет более 350 ...",Центральный административный округ,район Хамовники,"Комсомольский проспект, дом 28","Проезд - станция метро Фрунзенская, здание МДМ...","1 день - 650 руб., месяц - от 9000 руб.",NaN,Предприниматели,(985) 998-83-23,info@mkp-club.ru,www.delovar-cw.ru,"{type=Point, coordinates=[37.5794591899009, 55..."
2,23,Time Club Гнездо,4066282,Time Club Гнездо,Тайм клуб — это новый формат заведений длярабо...,Центральный административный округ,Басманный район,"Лялин переулок, строение 2",Вход с отдельного крыльца,"В зависимости от времени входа от 0,99 руб/мин...",NaN,Отраслевая специализация отсутствует,(495) 212-14-70,gnezdo@timeclub24.ru,www.timeclub24.ru/gnezdo,"{type=Point, coordinates=[37.651233441176615, ..."
3,24,Time Club Убежище,4066283,Time Club Убежище,Тайм клуб — это новый формат заведений длярабо...,Центральный административный округ,Басманный район,"Архангельский переулок, дом 7, строение 1",Вход с торца здания,"В зависимости от времени входа от 0,99 руб/мин...",NaN,Отраслевая специализация отсутствует,(495) 215-57-14,vault@timeclub24.ru,timeclub24.ru/ubezhische,"{type=Point, coordinates=[37.637629318889054, ..."
4,21,Time Club Re форма,4066284,Time Club Re форма,Тайм клуб — это новый формат заведений длярабо...,Центральный административный округ,Таганский район,"Георгиевский переулок, дом 1, строение 3",С Тверской поворот в арку. После входа варку и...,"В зависимости от времени входа от 0,99 руб/мин...",NaN,Отраслевая специализация отсутствует,(495) 212-91-14,reforma@timeclub24.ru,timeclub24.ru/reforma,"{type=Point, coordinates=[37.61343278955096, 5..."


# 2

Т.к. Если коворкинг бесплатный, то об этом сразу сообщат (ради рекламы), то нужно проверить есть ли в столбце "Информация о ценах" - найдём слова "Бесплатный" в данном столбце

In [3]:
coworking['Цена'] = coworking['Информация о ценах'].map(lambda x: str(x).lower())
coworking[coworking['Цена'].str.contains('бесплатно')].count()


Код                         10
Полное название             10
global_id                   10
Краткое название            10
Краткая характеристика      10
Административный округ      10
Район                       10
Адрес                       10
Уточнение местоположения     4
Информация о ценах          10
Год создания                 6
Отрасль                      6
Контактный телефон          10
Адрес электронной почты      6
Сайт                        10
Геоданные                   10
Цена                        10
dtype: int64

In [4]:
coworking[coworking['Цена'].str.contains('бесплатно')].count()['Код']/coworking.count()['Код']


0.23255813953488372

10 коворкингов могут быть бесплатными - 23%

In [5]:
coworking['Описание Цены'] = coworking['Цена'].str.replace('\d+', '')
coworking[(coworking['Описание Цены'] == 'бесплатно') | (coworking['Описание Цены'] == 'бесплатно.')].count()

Код                         7
Полное название             7
global_id                   7
Краткое название            7
Краткая характеристика      7
Административный округ      7
Район                       7
Адрес                       7
Уточнение местоположения    3
Информация о ценах          7
Год создания                5
Отрасль                     5
Контактный телефон          7
Адрес электронной почты     3
Сайт                        7
Геоданные                   7
Цена                        7
Описание Цены               7
dtype: int64

In [6]:
coworking[(coworking['Описание Цены'] == 'бесплатно') | (coworking['Описание Цены'] == 'бесплатно.')].count()['Код']/coworking.count()['Код']

0.16279069767441862

7 коворкингов полностью бесплатны - 16%

# 3

In [7]:
num = '0'
maxi = 0
for id_y in coworking['Цена']:
    id_x = id_y.split()
    for word in id_x:
        if word.isnumeric():
            num = num + word
        else:
            if float(num) > maxi:
                maxi = float(num)
                maxi_name = coworking[coworking['Цена'] == id_y]
            num = '0'
maxi_name

,Код,Полное название,global_id,Краткое название,Краткая характеристика,Административный округ,Район,Адрес,Уточнение местоположения,Информация о ценах,Год создания,Отрасль,Контактный телефон,Адрес электронной почты,Сайт,Геоданные,Цена,Описание Цены
40,44,CEO ROOMS coworking EMPIRE,656545634,CEO ROOMS EMPIRE,"Высокоскоростной интернет, принтер/сканер, офи...",Центральный административный округ,Пресненский район,"Пресненская набережная, д. 6, стр. 2","Москва-Сити, Пресненская наб., д. 6, стр.2, ба...",Офисы - от 160 000 / мес.\nРабочие места - от...,2017.0,NaN,(495) 249-90-90,info@ceorooms.ru,ceorooms.ru,"{type=Point, coordinates=[37.540200366974, 55....",офисы - от 160 000 / мес.\nрабочие места - от...,офисы - от / мес.\nрабочие места - от / ...


1) идем по строчкам коворкингов <br>
2) Делим вкладку цену по пробелам на список <br>
3) В каждом списке идем по словам, если слово число то начинаем составлять число<br>
4) Если следующее слово не число то:
- cохраняем строчку если она больше максимума<br>
- Если слово не число, то обнуляем его <br>
[3-4 пункты нужны т.к. встречаются числа написанные через пробел (и заменять replace - можно упустить что-либо)]

# 4A

In [8]:
title_list = []
for site in coworking['Сайт']:
    site_new = 'http://' + site
    req = requests.get(site_new,  headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 YaBrowser/17.10.0.2052 Yowser/2.5 Safari/537.36'})
    soup = BeautifulSoup(req.text, 'html.parser')
    if soup.title:
        print(soup.title.string)
        title_list.append(soup.title.string)
    else:
        title_list.append('None')
coworking['Title'] = pd.DataFrame({'Title': title_list})

Коворкинги "Union Place" в Москве, полноценные рабочие места и кабинеты.
None
Тайм клуб Гнездо ждет Вас! Антикафе Time Club Гнездо, Москва
Тайм Клуб Убежище работает круглосуточно. Антикафе Тайм Клуб Убежище, Москва
Time Club Re форма - новый формат антикафе! Сайт Тайм Клуба Реформа, Москва
404 Not Found
Рабочая Станция
Error 404: страница не найдена
Коворкинг «Свободное плавание» | Аренда рабочего места в офисе в Москве (метро Нагатинская)
Коворкинг-центр ЦУБ ТиНАО ГБУ «МБМ»
Pacific View Leadership Elementary • San Diego, CA - Главная | Facebook
Антикафе и коворкинг Хорошая Республика, официальный сайт, GoodRepublic — Антикафе и коворкинг Хорошая Республика, аренда рабочего места, помещения для дня рождения, вечеринки, для дня рождения и юбилея в Москве
Рабочая Станция
Коворкинг в Москве на Флаконе START HUB
FLACON дизайн-завод
Коворкинг
 CCI FABRIKA
Аренда офиса в Москве в бизнес центре MatrixOffice. Арендовать офис без посредника и без комиссии!
Циферблат Москва Покровка ул. Покровк

In [9]:
coworking['Title'] = pd.DataFrame({'Title': title_list})
coworking.head()

,Код,Полное название,global_id,Краткое название,Краткая характеристика,Административный округ,Район,Адрес,Уточнение местоположения,Информация о ценах,Год создания,Отрасль,Контактный телефон,Адрес электронной почты,Сайт,Геоданные,Цена,Описание Цены,Title
0,19,Коворкинг Union Place,4066280,Union Place,"Конференц-зал на 60 человек, переговорные комн...",Центральный административный округ,Таганский район,"1-й Гончарный переулок, д.3, стр.4","1-й Гончарный переулок, д.3, стр.4",Стоимость аренды конференц-зала 1500 рублей в ...,NaN,NaN,(495) 795-43-76,info@unionplace.ru,www.unionplace.ru,"{type=Point, coordinates=[37.65248776855666, 5...",стоимость аренды конференц-зала 1500 рублей в ...,стоимость аренды конференц-зала рублей в час;...,"Коворкинги ""Union Place"" в Москве, полноценные..."
1,20,Коворкинг и клубный офис клуба предпринимателе...,4066281,Коворкинг Деловар,"Клуб создан в 2003 году, объединяет более 350 ...",Центральный административный округ,район Хамовники,"Комсомольский проспект, дом 28","Проезд - станция метро Фрунзенская, здание МДМ...","1 день - 650 руб., месяц - от 9000 руб.",NaN,Предприниматели,(985) 998-83-23,info@mkp-club.ru,www.delovar-cw.ru,"{type=Point, coordinates=[37.5794591899009, 55...","1 день - 650 руб., месяц - от 9000 руб.","день - руб., месяц - от руб.",None
2,23,Time Club Гнездо,4066282,Time Club Гнездо,Тайм клуб — это новый формат заведений длярабо...,Центральный административный округ,Басманный район,"Лялин переулок, строение 2",Вход с отдельного крыльца,"В зависимости от времени входа от 0,99 руб/мин...",NaN,Отраслевая специализация отсутствует,(495) 212-14-70,gnezdo@timeclub24.ru,www.timeclub24.ru/gnezdo,"{type=Point, coordinates=[37.651233441176615, ...","в зависимости от времени входа от 0,99 руб/мин...","в зависимости от времени входа от , руб/мин.ес...",Тайм клуб Гнездо ждет Вас! Антикафе Time Club ...
3,24,Time Club Убежище,4066283,Time Club Убежище,Тайм клуб — это новый формат заведений длярабо...,Центральный административный округ,Басманный район,"Архангельский переулок, дом 7, строение 1",Вход с торца здания,"В зависимости от времени входа от 0,99 руб/мин...",NaN,Отраслевая специализация отсутствует,(495) 215-57-14,vault@timeclub24.ru,timeclub24.ru/ubezhische,"{type=Point, coordinates=[37.637629318889054, ...","в зависимости от времени входа от 0,99 руб/мин...","в зависимости от времени входа от , руб/мин.ес...",Тайм Клуб Убежище работает круглосуточно. Анти...
4,21,Time Club Re форма,4066284,Time Club Re форма,Тайм клуб — это новый формат заведений длярабо...,Центральный административный округ,Таганский район,"Георгиевский переулок, дом 1, строение 3",С Тверской поворот в арку. После входа варку и...,"В зависимости от времени входа от 0,99 руб/мин...",NaN,Отраслевая специализация отсутствует,(495) 212-91-14,reforma@timeclub24.ru,timeclub24.ru/reforma,"{type=Point, coordinates=[37.61343278955096, 5...","в зависимости от времени входа от 0,99 руб/мин...","в зависимости от времени входа от , руб/мин.ес...",Time Club Re форма - новый формат антикафе! Са...


1) итерируемся по сайтам <br>
2) Находим название и добавляем в список<br>
3) Добавляем список в дата-фрейм<br>

# 4Б

In [10]:
names_count = []
nums = 0
for site, names in zip(coworking['Сайт'], coworking['Полное название']):
    site_new = 'http://' + site
    req = requests.get(site_new,  headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 YaBrowser/17.10.0.2052 Yowser/2.5 Safari/537.36'})
    req = req.text
    for i in range(len(req) - len(names)):
        if req[i:i+len(names)] == names:
            nums += 1
    names_count.append(nums)
    nums = 0
coworking['Names_count'] = pd.DataFrame({'Names_count': names_count})

In [11]:
coworking.head()

,Код,Полное название,global_id,Краткое название,Краткая характеристика,Административный округ,Район,Адрес,Уточнение местоположения,Информация о ценах,Год создания,Отрасль,Контактный телефон,Адрес электронной почты,Сайт,Геоданные,Цена,Описание Цены,Title,Names_count
0,19,Коворкинг Union Place,4066280,Union Place,"Конференц-зал на 60 человек, переговорные комн...",Центральный административный округ,Таганский район,"1-й Гончарный переулок, д.3, стр.4","1-й Гончарный переулок, д.3, стр.4",Стоимость аренды конференц-зала 1500 рублей в ...,NaN,NaN,(495) 795-43-76,info@unionplace.ru,www.unionplace.ru,"{type=Point, coordinates=[37.65248776855666, 5...",стоимость аренды конференц-зала 1500 рублей в ...,стоимость аренды конференц-зала рублей в час;...,"Коворкинги ""Union Place"" в Москве, полноценные...",0
1,20,Коворкинг и клубный офис клуба предпринимателе...,4066281,Коворкинг Деловар,"Клуб создан в 2003 году, объединяет более 350 ...",Центральный административный округ,район Хамовники,"Комсомольский проспект, дом 28","Проезд - станция метро Фрунзенская, здание МДМ...","1 день - 650 руб., месяц - от 9000 руб.",NaN,Предприниматели,(985) 998-83-23,info@mkp-club.ru,www.delovar-cw.ru,"{type=Point, coordinates=[37.5794591899009, 55...","1 день - 650 руб., месяц - от 9000 руб.","день - руб., месяц - от руб.",None,0
2,23,Time Club Гнездо,4066282,Time Club Гнездо,Тайм клуб — это новый формат заведений длярабо...,Центральный административный округ,Басманный район,"Лялин переулок, строение 2",Вход с отдельного крыльца,"В зависимости от времени входа от 0,99 руб/мин...",NaN,Отраслевая специализация отсутствует,(495) 212-14-70,gnezdo@timeclub24.ru,www.timeclub24.ru/gnezdo,"{type=Point, coordinates=[37.651233441176615, ...","в зависимости от времени входа от 0,99 руб/мин...","в зависимости от времени входа от , руб/мин.ес...",Тайм клуб Гнездо ждет Вас! Антикафе Time Club ...,12
3,24,Time Club Убежище,4066283,Time Club Убежище,Тайм клуб — это новый формат заведений длярабо...,Центральный административный округ,Басманный район,"Архангельский переулок, дом 7, строение 1",Вход с торца здания,"В зависимости от времени входа от 0,99 руб/мин...",NaN,Отраслевая специализация отсутствует,(495) 215-57-14,vault@timeclub24.ru,timeclub24.ru/ubezhische,"{type=Point, coordinates=[37.637629318889054, ...","в зависимости от времени входа от 0,99 руб/мин...","в зависимости от времени входа от , руб/мин.ес...",Тайм Клуб Убежище работает круглосуточно. Анти...,10
4,21,Time Club Re форма,4066284,Time Club Re форма,Тайм клуб — это новый формат заведений длярабо...,Центральный административный округ,Таганский район,"Георгиевский переулок, дом 1, строение 3",С Тверской поворот в арку. После входа варку и...,"В зависимости от времени входа от 0,99 руб/мин...",NaN,Отраслевая специализация отсутствует,(495) 212-91-14,reforma@timeclub24.ru,timeclub24.ru/reforma,"{type=Point, coordinates=[37.61343278955096, 5...","в зависимости от времени входа от 0,99 руб/мин...","в зависимости от времени входа от , руб/мин.ес...",Time Club Re форма - новый формат антикафе! Са...,5


In [12]:
coworking[coworking['Names_count'] == coworking['Names_count'].max()]

,Код,Полное название,global_id,Краткое название,Краткая характеристика,Административный округ,Район,Адрес,Уточнение местоположения,Информация о ценах,Год создания,Отрасль,Контактный телефон,Адрес электронной почты,Сайт,Геоданные,Цена,Описание Цены,Title,Names_count
18,13,Циферблат,4066299,Циферблат,"В Циферблате три зала. Здесь можно работать, о...",Центральный административный округ,Басманный район,"улица Покровка, дом 12 строение 1",NaN,"3 руб. за минуту в первый час, далее 2 руб., п...",2011.0,Отраслевая специализация отсутствует,(962) 964-67-86,meetinziferblat@gmail.com,pokrovka.ziferblat.net,"{type=Point, coordinates=[37.64462602668295, 5...","3 руб. за минуту в первый час, далее 2 руб., п...","руб. за минуту в первый час, далее руб., пос...","Циферблат Москва Покровка ул. Покровка, д. 12 ...",19


Такой же алгоритм, как и в 4А, но добавляем счетчик упоминаний в тексте. Находим самый частый простым максимум

# 4В

In [13]:
link_num = 0
link_count = []
for site, names in zip(coworking['Сайт'], coworking['Полное название']):
    site_new = 'http://' + site
    req = requests.get(site_new,  headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 YaBrowser/17.10.0.2052 Yowser/2.5 Safari/537.36'})
    soup = BeautifulSoup(req.text, 'html.parser')
    linkList = []
    for link in soup.findAll('a'):
        linkText = link.get('href')
        if linkText and 'http' in linkText and site[0:site.find('/')] not in linkText:
                link_num += 1
    link_count.append(link_num)
    link_num = 0
coworking['Link_count'] = pd.DataFrame({'Link_count': link_count})

In [14]:
coworking[coworking['Link_count'] == coworking['Link_count'].max()]

,Код,Полное название,global_id,Краткое название,Краткая характеристика,Административный округ,Район,Адрес,Уточнение местоположения,Информация о ценах,...,Отрасль,Контактный телефон,Адрес электронной почты,Сайт,Геоданные,Цена,Описание Цены,Title,Names_count,Link_count
16,10,Центр Творческих Индустрий «Фабрика»,4066296,ЦТИ Фабрика,Под крышей ФАБРИКИ гармонично соседствуют маст...,Центральный административный округ,Басманный район,"Переведеновский переулок, дом 18",NaN,NaN,...,NaN,(499) 265-39-26,info@fabrikacci.com,www.fabrikacci.com,"{type=Point, coordinates=[37.68973034581759, 5...",nan,nan,CCI FABRIKA,1,60


In [15]:
coworking[coworking['Link_count'] == coworking['Link_count'].min()]

,Код,Полное название,global_id,Краткое название,Краткая характеристика,Административный округ,Район,Адрес,Уточнение местоположения,Информация о ценах,...,Отрасль,Контактный телефон,Адрес электронной почты,Сайт,Геоданные,Цена,Описание Цены,Title,Names_count,Link_count
5,22,Пространственно-временной клуб Коперник Антикафе,4066285,ПВК Коперник Антикафе,Пространственно-временной клуб Коперник - уник...,Южный административный округ,Даниловский район,"Улица Мытная, дом 52",10 мин от м.Тульская/ 8 мин от м.Шаболовская,Тариф антикафе - 2р минута. Для вступивших в г...,...,NaN,(903) 790-09-29,pvk222@mail.ru,vk.com/pvk_kopernik\n,"{type=Point, coordinates=[37.61856679123033, 5...",тариф антикафе - 2р минута. для вступивших в г...,тариф антикафе - р минута. для вступивших в гр...,404 Not Found,0,0
31,34,FRIENDZONA.CLUB,64874637,Антикафе «Friend Zona»,Принтер\nWi-Fi Free\nФлипчарт\nРозетки\nИгры\n...,Северный административный округ,Головинский район,"Головинское шоссе, д. 4",Friendzona.club удобно расположен в двух шагах...,Стоимость прибывания в антикафе friedzona.club...,...,NaN,(985) 589-35-97,fzmsk@yandex.ru,www.friendzona.club,"{type=Point, coordinates=[37.48940143025633, 5...",стоимость прибывания в антикафе friedzona.club...,стоимость прибывания в антикафе friedzona.club...,,0,0
35,39,Коворкинг на Рижской,612740697,Коворкинг на Рижской,В Вашем распоряжении – 2 изолированные стеклян...,Северо-Восточный административный округ,район Марьина Роща,"Москва, Рижская площадь, д.9, стр. 2",Расположен в 5 минутах от метро и 10 минутах о...,1 посещение 500 р. (действует 1 день)\n7 посещ...,...,NaN,(499) 266-81-27,admin@coworiga.ru,coworiga.ru,"{type=Point, coordinates=[37.63187053036785, 5...",1 посещение 500 р. (действует 1 день)\n7 посещ...,посещение р. (действует день)\n посещений ...,Коворкинг на рижской,0,0


In [16]:
coworking[coworking['Link_count'] == 1]

,Код,Полное название,global_id,Краткое название,Краткая характеристика,Административный округ,Район,Адрес,Уточнение местоположения,Информация о ценах,...,Отрасль,Контактный телефон,Адрес электронной почты,Сайт,Геоданные,Цена,Описание Цены,Title,Names_count,Link_count
19,14,ANTICAFE BABOCHKI,4066300,Бабочки,Антикафе «Бабочки» - это пространство с отдель...,Центральный административный округ,Таганский район,"Таганская улица, дом 29, строение 1",NaN,Тариф «Рабочий» - неограниченное посещение ант...,...,Отраслевая специализация отсутствует,(495) 728-21-61;(926) 580-22-20,NaN,www.anticafebabochki.ru,"{type=Point, coordinates=[37.66733028875172, 5...",тариф «рабочий» - неограниченное посещение ант...,тариф «рабочий» - неограниченное посещение ант...,None,0,1
20,15,Антикафе «CleverClub»,4066301,CleverClub,Преимущества коворкинга: расположение в центре...,Центральный административный округ,район Якиманка,"улица Малая Полянка, дом 7, строение 5",NaN,"2руб. - 1 мин., пон.- пят.---топ/чек(10.00-23...",...,Отраслевая специализация отсутствует,(966) 195-49-04;(965) 443-66-03,manager@clever-c.com,clever-c.com/coworking/,"{type=Point, coordinates=[37.617486806425546, ...","2руб. - 1 мин., пон.- пят.---топ/чек(10.00-23...","руб. - мин., пон.- пят.---топ/чек(.-.)--руб....",Clever-c.com,0,1
21,16,Коворкинг «на Серпуховской»,4066302,На Серпуховской,Удобное месторасположение нашего офисного цент...,Центральный административный округ,район Замоскворечье,"1-й Щипковский переулок, дом 20",NaN,"1 день - 800 рублей, 1 неделя - 3400 рублей, 1...",...,Отраслевая специализация отсутствует,(985) 193-12-71,klass@indarenda.ru,indarenda.ru/bokovoe-menyu/prozhivanie/,"{type=Point, coordinates=[37.63257841275251, 5...","1 день - 800 рублей, 1 неделя - 3400 рублей, 1...","день - рублей, неделя - рублей, месяц - ...",Индустрия недвижимости Учебный центр,0,1


1) Находим все ссылки внешние - используем BS + смотрим, чтобы ссылка не была частью того сайта (для этого ищем "/") <br>
2) Таким же счетчиком находим<br>
3) Смотрим минимум (0 и 1, т.к. не у всех есть рабочие сайты